# How to create a source

This tutorial demonstrates how to create a source.

There are different type of sources available: luminaire source, surface source, ray file source.

## Prerequisites

### Perform imports

In [1]:
from pathlib import Path

from ansys.speos.core import GeoRef, Project, Speos, launcher
from ansys.speos.core.source import (
    SourceLuminaire,
    SourceRayFile,
    SourceSurface,
)


### Define constants
Constants help ensure consistency and avoid repetition throughout the example.

In [2]:
HOSTNAME = "localhost"
GRPC_PORT = 50098  # Be sure the Speos GRPC Server has been started on this port.
USE_DOCKER = True  # Set to False if you're running this example locally as a Notebook.
IES = "IES_C_DETECTOR.ies"

### Define helper functions

In [3]:
def create_helper_geometries(project: Project):
    """Create bodies and faces."""

    def create_face(body):
        (
            body.create_face(name="TheFaceF")
            .set_vertices([0, 0, 0, 1, 0, 0, 0, 1, 0])
            .set_facets([0, 1, 2])
            .set_normals([0, 0, 1, 0, 0, 1, 0, 0, 1])
            .commit()
        )

    root_part = project.create_root_part().commit()
    body_b1 = root_part.create_body(name="TheBodyB").commit()
    body_b2 = root_part.create_body(name="TheBodyC").commit()
    body_b3 = root_part.create_body(name="TheBodyD").commit()
    body_b4 = root_part.create_body(name="TheBodyE").commit()
    for b in [body_b1, body_b2, body_b3, body_b4]:
        create_face(b)

## Model Setup

### Load assets
The assets used to run this example are available in the
[PySpeos repository](https://github.com/ansys/pyspeos/) on GitHub.

> **Note:** Make sure you
> have downloaded simulation assets and set ``assets_data_path``
> to point to the assets folder.

In [4]:
if USE_DOCKER:  # Running on the remote server.
    assets_data_path = Path("/app") / "assets"
else:
    assets_data_path = Path("/path/to/your/download/assets/directory")

### Connect to the RPC Server
This Python client connects to a server where the Speos engine
is running as a service. In this example, the server and
client are the same machine. The launch_local_speos_rpc_method can
be used to start a local instance of the service.

In [5]:
if USE_DOCKER:
    speos = Speos(host=HOSTNAME, port=GRPC_PORT)
else:
    speos = launcher.launch_local_speos_rpc_server(port=GRPC_PORT)

### Create a new project

The only way to create a source using the core layer, is to create it from a project.
The ``Project`` class is instantiated by passing a ``Speos`` instance

In [6]:
p = Project(speos=speos)
print(p)

{
    "name": "",
    "description": "",
    "metadata": {},
    "part_guid": "",
    "sources": [],
    "sensors": [],
    "simulations": [],
    "materials": [],
    "scenes": []
}


### Source Creation

**Create locally:**
The mention "local: " is added when printing the source data and information is not yet
pushed to the RPC server

In [7]:
intensity_file_path = str(assets_data_path / IES)
source1 = p.create_source(name="Luminaire.1", feature_type=SourceLuminaire)  # type luminaire
source1.set_intensity_file_uri(uri=intensity_file_path)
print(source1)

local: {
    "name": "Luminaire.1",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "flux_from_intensity_file": {},
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "",
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


**Push it to the server.**

After it is committed to the server, the mention "local: " is no more present when printing the
source.

In [8]:
source1.commit()
print(source1)

{
    "name": "Luminaire.1",
    "metadata": {
        "UniqueId": "1b5e86e9-a11c-4b23-a7e2-5a659ac49558"
    },
    "source_guid": "17e878e1-69c1-4c8d-b1fb-58b3bac777ac",
    "description": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "flux_from_intensity_file": {},
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "d1f62f4b-e9e7-4f8c-9712-c34b35b1b70f",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "incandescent": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
    

**Changing additional Source Properties**

Setting several more characteristics.

In [9]:
intensity_file_path = str(assets_data_path / IES)
source2 = p.create_source(name="Luminaire.2", feature_type=SourceLuminaire)
source2.set_intensity_file_uri(uri=intensity_file_path)
source2.set_flux_radiant()  # select flux radiant with default value
# choose the source location [Origin, Xvector, Yvector, Zvector]
source2.set_axis_system(axis_system=[20, 50, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])
source2.set_spectrum().set_blackbody()  # choose blackbody with default value for the spectrum
source2.commit()  # Push to the server
print(source2)

{
    "name": "Luminaire.2",
    "metadata": {
        "UniqueId": "90b72945-1b1f-40ac-b050-cc2159a0c07a"
    },
    "source_guid": "870bad75-6891-490b-aabb-4eb63df17e19",
    "description": "",
    "source": {
        "name": "Luminaire.2",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.0
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "f0a5fe8c-76f4-4911-bebc-176b14acae28",
            "spectrum": {
                "name": "Luminaire.2.Spectrum",
                "blackbody": {
                    "temperature": 2856.0
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                20.0,
                50.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
      

**Source Instance**

As mention "local: " is added if it is not yet committed to the server.

In [10]:
print(source1)

{
    "name": "Luminaire.1",
    "metadata": {
        "UniqueId": "1b5e86e9-a11c-4b23-a7e2-5a659ac49558"
    },
    "source_guid": "17e878e1-69c1-4c8d-b1fb-58b3bac777ac",
    "description": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "flux_from_intensity_file": {},
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "d1f62f4b-e9e7-4f8c-9712-c34b35b1b70f",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "incandescent": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
    

**Project:**

Committed feature will appear inside the project information.

In [11]:
print(p)

{
    "sources": [
        {
            "name": "Luminaire.1",
            "metadata": {
                "UniqueId": "1b5e86e9-a11c-4b23-a7e2-5a659ac49558"
            },
            "source_guid": "17e878e1-69c1-4c8d-b1fb-58b3bac777ac",
            "description": "",
            "source": {
                "name": "Luminaire.1",
                "luminaire": {
                    "flux_from_intensity_file": {},
                    "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
                    "spectrum_guid": "d1f62f4b-e9e7-4f8c-9712-c34b35b1b70f",
                    "spectrum": {
                        "name": "Luminaire.1.Spectrum",
                        "predefined": {
                            "incandescent": {}
                        },
                        "description": "",
                        "metadata": {}
                    },
                    "axis_system": [
                        0.0,
                        0.0,
                        0.0

**Update:**

> **Note:** If you are manipulating a source already committed, don't forget to commit your
> changes.
> If you don't, you will still only watch what is committed on the server.

In [12]:
source1.set_flux_radiant(value=1.2)  # modify radiant flux value
source1.set_axis_system(axis_system=[17, 10, 10, 1, 0, 0, 0, 1, 0, 0, 0, 1])  # modify axis system
source1.set_spectrum().set_halogen()  # modify spectrum by choosing halogen
source1.commit()  # Push changes to the server
print(source1)

{
    "name": "Luminaire.1",
    "metadata": {
        "UniqueId": "1b5e86e9-a11c-4b23-a7e2-5a659ac49558"
    },
    "source_guid": "17e878e1-69c1-4c8d-b1fb-58b3bac777ac",
    "description": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.2
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "d1f62f4b-e9e7-4f8c-9712-c34b35b1b70f",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "halogen": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                17.0,
                10.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
             

**Reset**

Possibility to reset local values from the one available in the server.

In [13]:
source1.set_flux_luminous()  # modify to luminous flux BUT no commit
source1.reset()
# reset -> this will apply the server value to the local value the local value will be back to
# halogen
source1.delete()  # delete (to display the local value with the below print)
print(source1)

local: {
    "name": "Luminaire.1",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Luminaire.1",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.2
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "d1f62f4b-e9e7-4f8c-9712-c34b35b1b70f",
            "spectrum": {
                "name": "Luminaire.1.Spectrum",
                "predefined": {
                    "halogen": {}
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                17.0,
                10.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


**Delete**

Once the data is deleted from the server, you can still work with local data and maybe commit
later.

In [14]:
source2.delete()
print(source2)
source1.delete()
print(p)

local: {
    "name": "Luminaire.2",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Luminaire.2",
        "luminaire": {
            "radiant_flux": {
                "radiant_value": 1.0
            },
            "intensity_file_uri": "/app/assets/IES_C_DETECTOR.ies",
            "spectrum_guid": "f0a5fe8c-76f4-4911-bebc-176b14acae28",
            "spectrum": {
                "name": "Luminaire.2.Spectrum",
                "blackbody": {
                    "temperature": 2856.0
                },
                "description": "",
                "metadata": {}
            },
            "axis_system": [
                20.0,
                50.0,
                10.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    

## Other Sources Examples

### Ray-file source

In [15]:
ray_file_path = str(assets_data_path / "Rays.ray")

source3 = p.create_source(name="Ray-file.1", feature_type=SourceRayFile)  # type ray file
source3.set_ray_file_uri(uri=ray_file_path)
source3.commit()
print(source3)

{
    "name": "Ray-file.1",
    "metadata": {
        "UniqueId": "f48edbf5-0667-48ab-9634-1d2981e3eb17"
    },
    "source_guid": "3dfd4318-5a69-4aee-9fc9-ad570f355eb4",
    "description": "",
    "source": {
        "name": "Ray-file.1",
        "rayfile": {
            "ray_file_uri": "/app/assets/Rays.ray",
            "flux_from_ray_file": {},
            "spectrum_from_ray_file": {},
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


In [16]:
source3.set_flux_luminous()
source3.commit()
print(source3)

{
    "name": "Ray-file.1",
    "metadata": {
        "UniqueId": "f48edbf5-0667-48ab-9634-1d2981e3eb17"
    },
    "source_guid": "3dfd4318-5a69-4aee-9fc9-ad570f355eb4",
    "description": "",
    "source": {
        "name": "Ray-file.1",
        "rayfile": {
            "ray_file_uri": "/app/assets/Rays.ray",
            "luminous_flux": {
                "luminous_value": 683.0
            },
            "spectrum_from_ray_file": {},
            "axis_system": [
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0,
                0.0,
                0.0,
                0.0,
                1.0
            ]
        },
        "description": "",
        "metadata": {}
    }
}


In [17]:
source3.delete()

### Surface source

In [18]:
create_helper_geometries(p)
source4 = p.create_source(name="Surface.1", feature_type=SourceSurface)
source4.set_exitance_constant(
    geometries=[
        (GeoRef.from_native_link("TheBodyB/TheFaceF"), False),
        (GeoRef.from_native_link("TheBodyC/TheFaceF"), True),
    ]
)
source4.commit()
print(source4)

{
    "name": "Surface.1",
    "metadata": {
        "UniqueId": "01f1d742-74b7-4ef8-8dee-c3a32467baa6"
    },
    "source_guid": "3727a11b-1208-4f9d-a5fe-42d98f28627c",
    "description": "",
    "source": {
        "name": "Surface.1",
        "surface": {
            "luminous_flux": {
                "luminous_value": 683.0
            },
            "intensity_guid": "7225063f-212f-417f-be62-e8f5a8f066ce",
            "exitance_constant": {
                "geo_paths": [
                    {
                        "geo_path": "TheBodyB/TheFaceF",
                        "reverse_normal": false
                    },
                    {
                        "geo_path": "TheBodyC/TheFaceF",
                        "reverse_normal": true
                    }
                ]
            },
            "spectrum_guid": "213f21fb-d068-4377-a235-9fb635a162af",
            "intensity": {
                "name": "Surface.1.Intensity",
                "cos": {
                    

In [19]:
source4.set_flux_luminous_intensity()
source4.set_intensity().set_gaussian().set_axis_system(
    axis_system=[10, 50, 20, 1, 0, 0, 0, 1, 0, 0, 0, 1]
)
source4.commit()
print(source4)

{
    "name": "Surface.1",
    "metadata": {
        "UniqueId": "01f1d742-74b7-4ef8-8dee-c3a32467baa6"
    },
    "source_guid": "3727a11b-1208-4f9d-a5fe-42d98f28627c",
    "description": "",
    "source": {
        "name": "Surface.1",
        "surface": {
            "luminous_intensity_flux": {
                "luminous_intensity_value": 5.0
            },
            "intensity_guid": "7225063f-212f-417f-be62-e8f5a8f066ce",
            "exitance_constant": {
                "geo_paths": [
                    {
                        "geo_path": "TheBodyB/TheFaceF",
                        "reverse_normal": false
                    },
                    {
                        "geo_path": "TheBodyC/TheFaceF",
                        "reverse_normal": true
                    }
                ]
            },
            "spectrum_guid": "213f21fb-d068-4377-a235-9fb635a162af",
            "intensity": {
                "name": "Surface.1.Intensity",
                "gaussian":

In [20]:
source4.delete()
print(source4)

local: {
    "name": "Surface.1",
    "description": "",
    "metadata": {},
    "source_guid": "",
    "source": {
        "name": "Surface.1",
        "surface": {
            "luminous_intensity_flux": {
                "luminous_intensity_value": 5.0
            },
            "intensity_guid": "7225063f-212f-417f-be62-e8f5a8f066ce",
            "exitance_constant": {
                "geo_paths": [
                    {
                        "geo_path": "TheBodyB/TheFaceF",
                        "reverse_normal": false
                    },
                    {
                        "geo_path": "TheBodyC/TheFaceF",
                        "reverse_normal": true
                    }
                ]
            },
            "spectrum_guid": "213f21fb-d068-4377-a235-9fb635a162af",
            "intensity": {
                "name": "Surface.1.Intensity",
                "gaussian": {
                    "FWHM_angle_x": 30.0,
                    "FWHM_angle_y": 30.0,
      

When creating sources, this creates some intermediate objects (spectrums, intensity templates).

Deleting a source does not delete in cascade those objects
because they could be used by some other entities from core layer.

Then at the end of the example, we just clean all databases

In [21]:
for item in speos.client.intensity_templates().list() + speos.client.spectrums().list():
    item.delete()

speos.close()

True